## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")


In [3]:
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,North Bend,43.4065,-124.2243,52.86,85,75,17.27,broken clouds,US
1,1,Vaini,-21.2000,-175.2000,82.56,83,75,6.91,broken clouds,TO
2,2,Jamestown,42.0970,-79.2353,54.99,49,0,10.36,clear sky,US
3,3,Komsomolskiy,40.4272,71.7189,51.22,87,100,4.61,mist,UZ
4,4,Makakilo City,21.3469,-158.0858,79.12,53,0,10.36,clear sky,US


In [4]:
city_data_df.shape

(551, 10)

In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
print(f'Mimunum Temperature Selected :{min_temp}')
print(f'Maximum Temperature Selected :{max_temp}')

Mimunum Temperature Selected :75.0
Maximum Temperature Selected :85.0


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_data = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                 (city_data_df["Max Temp"] <= max_temp)]

In [21]:
# 4a. Determine if there are any empty rows.

filtered_city_data.count()

City_ID                142
City                   142
Lat                    142
Lng                    142
Max Temp               142
Humidity               142
Cloudiness             142
Wind Speed             142
Current Description    142
Country                142
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = filtered_city_data.dropna()

In [23]:
clean_df.count()

City_ID                142
City                   142
Lat                    142
Lng                    142
Max Temp               142
Humidity               142
Cloudiness             142
Wind Speed             142
Current Description    142
Country                142
dtype: int64

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
4,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,
7,Bengkulu,ID,77.92,light rain,-3.8004,102.2655,
8,Buchanan,LR,79.07,clear sky,5.8808,-10.0467,
11,Puerto Colombia,CO,80.69,broken clouds,10.9878,-74.9547,
20,Atuona,PF,77.74,light rain,-9.8000,-139.0333,
22,Puerto Ayora,EC,77.56,broken clouds,-0.7393,-90.3518,
25,Salina Cruz,MX,81.28,clear sky,16.1667,-95.2000,
26,Rikitea,PF,79.79,few clouds,-23.1203,-134.9692,
27,Hithadhoo,MV,82.29,clear sky,-0.6000,73.0833,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index,row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 

    hotels = requests.get(base_url, params= params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
#There is no row to Drop 
hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
4,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club
7,Bengkulu,ID,77.92,light rain,-3.8004,102.2655,Grage Hotel Bengkulu
8,Buchanan,LR,79.07,clear sky,5.8808,-10.0467,Kamaneahn Hotel Inc
11,Puerto Colombia,CO,80.69,broken clouds,10.9878,-74.9547,Hotel El Emigrante


In [27]:
# Ther is No Nans
hotel_df.count()

City                   142
Country                142
Max Temp               142
Current Description    142
Lat                    142
Lng                    142
Hotel Name             142
dtype: int64

In [28]:
clean_hotel_df = hotel_df

In [29]:
clean_hotel_df.shape

(142, 7)

In [30]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name<b></dt><dd>{Hotel Name}</dd>
<dt><b>City<b></dt><dd>{City}</dd>
<dt><b>Country<b></dt><dd>{Country}</dd>
<dt><b>Current Weather<b></dt><dd> {Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5 , layout={'height': '700px'})
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='700px'))